# code for learning lol

##### as always the chatbot will work by 1st: doing an NLP stuff, 2nd: doing some Machine Learning stuff...

##### the nlp will be done by
##### 1- tokenizing
##### 2- stemming/lemmatizing and lowering
##### 3- excluding punctioation characters
##### 4- throw the result in a bag of word 

In [1]:
import json
import pandas as pd
from nltk.stem import WordNetLemmatizer, PorterStemmer
import nltk
import numpy as np
import random

In [2]:
intents = json.loads(open('intents.json').read())

In [3]:
# a bag of words is a bunch of sentances turned into 0s and 1s so we can determine how many times it has occured
# example
# the tag here is greeting
# ['hello', 'hey', 'how', 'are', 'you']
# [1, 1, 0, 0, 0]
# ['are', 'you', 'a', 'robot', 'hello']
# [0, 0, 0, 0, 1]

In [4]:
lem = WordNetLemmatizer()
stm = PorterStemmer()

In [5]:
def tokenize(sentance):
    return nltk.word_tokenize(sentance)

def lemm(word):
    return lem.lemmatize(word.lower())

def stem(word):
    return stm.stem(word.lower())

# this is probably the most important function, try to understand what it does
def bag_of_words(tokenized_sentance, all_words):
    tokenized_sentance = [lemm(word) for word in tokenized_sentance]
    bag = np.zeros(len(all_words), dtype=np.float32)
    for index_of_the_word, word in enumerate(all_words):
        if word in tokenized_sentance:
            bag[index_of_the_word] = 1.0
    return bag

In [6]:
sentance = "this is bullshit I can't believe that this is how you do natural language processing, I thought there might be more steps!"

#### here we added the tags in the josn file into a list 
#### we've also made a nested loop to tokenize the patterns in each tag and add it to 'all_words' list
#### so it will come to one tag, tokenize it's patterns, and saves it to the 'all_words' list and then go to another tag
#### after we did all of that, we then made a list of tuples called xy, the list will work like this

In [7]:
#[
#([a tokenized sentance], the tag it's asscoiated with), 
# ([another tokenized sentance], the other tag that is associated),
# (['hello', 'how', 'are', 'you'], 'greeting'),
# (['how', 'old', 'are', 'you'], 'age')
#]

In [8]:
all_words = []
tags =[]
xy = []

for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        
        # notice here that we have uesd extend, becaues append will add each tokenized pattern as an indvidual list and we don't want that,
        # we want a 1D array not a multidimensional array
        # append -> ['google'] + ['hello'] = [['google'], ['hello']]
        # extedn -> ['google'] + ['hello'] = ['google', 'hello']
        
        all_words.extend(w)
        xy.append((w, tag))

# removing punctioation marks
ignore_words = [',', '.', '!', '?']
clean_words = [word for word in all_words if word not in ignore_words]

# lowering words
lower_words = [word.lower() for word in clean_words]

# sorting words
sortd_words = sorted(set(lower_words))

# we sorted the tags so we can have an easier job when we do the next step
lower_tags = [tag.lower() for tag in tags]
sorted_tags = sorted(lower_tags)

In [9]:
print(sortd_words)

["'m", "'s", '10', '19', 'a', 'age', 'am', 'anyone', 'are', 'ask', 'awesome', 'bad', 'bbye', 'be', 'best', 'bye', 'can', 'contact', 'could', 'covid', 'creator', 'cricket', 'current', 'date', 'day', 'designed', 'developer', 'do', 'doing', 'dumb', 'fine', 'for', 'funny', 'get', 'good', 'goodbye', 'google', 'great', 'haha', 'he', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'hot', 'how', 'i', 'idiot', 'india', 'inspiration', 'inspires', 'internet', 'is', 'it', 'joke', 'karan', 'know', 'later', 'latest', 'laugh', 'lmao', 'lol', 'lost', 'made', 'make', 'malik', 'matches', 'me', 'motivates', 'namaste', 'news', 'next', 'nice', 'no', 'nope', 'offered', 'ok', 'old', 'programmed', 'programmer', 'provide', 'question', 'riddle', 'rofl', 'score', 'search', 'see', 'set', 'shut', 'songs', 'suggest', 'suggestions', 'sup', 'support', 'talk', 'talking', 'tell', 'temperature', 'ten', 'thank', 'thanks', 'that', 'thats', 'the', 'there', 'till', 'time', 'timer', 'to', 'today', 'top', 'up', 'u

In [10]:
print(sorted_tags)

['activity', 'age', 'appreciate', 'contact', 'covid19', 'cricket', 'datetime', 'exclaim', 'goodbye', 'google', 'greeting', 'greetreply', 'haha', 'identity', 'inspire', 'insult', 'jokes', 'karan', 'news', 'nicetty', 'no', 'noanswer', 'options', 'programmer', 'riddle', 'song', 'suggest', 'thanks', 'timer', 'weather', 'whatsup']


# after all that mumbo jumbo let's create a bag of words

### we use the all_words list just as a reference so we can give each word a BoW
### let's imagine this is the current BoW

In [11]:
# words -> ['hello', 'bye', 'thanks', 'time']
# bow_words -> [1,1,1,1]
# random_tokenized_string -> ['hello', 'my', 'dear']
# throwing the string in the bow it will give us
# [1,0,0,0]
# we can figure out that this string is greating since it match the 'hello', in the bow
# this is all according to my understanding

In [12]:
x_train = []
y_train =[]

for (pattern_sentance, tag) in xy:
    bag = bag_of_words(pattern_sentance, all_words)
    x_train.append(bag)

    label = tags.index(tag)
    y_train.append(label) # this is basiclly one_hot_encoding the tags or as it called in pytorch CrossEntropyLoss

x_train = np.array(x_train)
y_train = np.array(y_train)
y_train = y_train.astype('longlong')

# and now for some PyTorch

In [13]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [14]:
class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(x_train)
        self.x_data = x_train
        self.y_data = y_train
       
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.n_samples

In [15]:
# hyperparameters
batch_size = 8

In [16]:
dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

### we will want to create a new class for the neural network part of the chatbot, the first class was for the data, and now this one is for the machine learning

In [17]:
# this little bad boy is our model

In [18]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation, no selfmax
        return out

In [19]:
# more hyperparameters
hidden_size = 8
output_size = len(tags)
input_size = len(x_train[0])
learning_rate = 0.001
num_epochs = 1000

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNet(input_size, hidden_size, output_size).to(device)

In [21]:
# loss and optimize
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [22]:
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(device)
        # forward
        output = model(words)
        loss = criterion(output, labels)

        # backward and optimizer steps
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'epoch: {epoch+1}/{num_epochs}, loss = {loss.item():.4f}')
print(f'final loss, loss = {loss.item():.4f}')

epoch: 100/1000, loss = 1.6618
epoch: 200/1000, loss = 0.0211
epoch: 300/1000, loss = 0.0234
epoch: 400/1000, loss = 0.0014
epoch: 500/1000, loss = 0.0001
epoch: 600/1000, loss = 0.0031
epoch: 700/1000, loss = 0.0432
epoch: 800/1000, loss = 0.0344
epoch: 900/1000, loss = 0.0000
epoch: 1000/1000, loss = 0.0000
final loss, loss = 0.0000


In [23]:
# saving the file

In [24]:
data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "output_size": output_size,
    "hidden_size": hidden_size,
    "all_words": all_words,
    "tags": tags
}

In [25]:
FILE = "data.pth"

In [26]:
torch.save(data, FILE)

In [27]:
print(f'training complete and file saved in {FILE}')

training complete and file saved in data.pth


In [28]:
# there are 4 python files in this project but I've placed all of their functions inside this notebook
# intents.json
# train.py
# chat.py
# model.py
# nltk_utils.py

# building the chatbot

In [29]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
with open('intents.json', 'r') as f:
    intetns = json.load(f)
    
FILE = 'data.pth'
data = torch.load(FILE)

input_size = data['input_size']
hidden_size = data['hidden_size']
output_size = data['output_size']
all_words = data['all_words']
tags = data['tags']
model_state = data['model_state']

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

NeuralNet(
  (l1): Linear(in_features=276, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=31, bias=True)
  (relu): ReLU()
)

In [33]:
bot_name = 'Bot'
print("let's chat!, type quit to exit")
while True:
    sentance = input("You: ")
    if sentance=='quit':
        break
    sentance = tokenize(sentance)
    x = bag_of_words(sentance, all_words)
    x = x.reshape(1, x.shape[0])
    x = torch.from_numpy(x)

    output = model(x)
    _, predicted = torch.max(output, dim=1)
    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]

    if prob.item() > 0.4:
        for intent in intetns['intents']:
            if tag==intent['tag']:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f'{bot_name}: I do not understand')

let's chat!, type quit to exit


KeyboardInterrupt: Interrupted by user

In [40]:
file = json.loads(open('written_intents.json').read())
df = pd.json_normalize(file['intents'])

In [41]:
df

,tag,patterns,responses
0,greeting,"[hello, hi, hey, what's up, anybody here, good...","[Hello!, Hi there, how can I help you today?, ..."
1,goodbye,"[good bye, adios, see you later, bye]","[Come back soon!, Good bye!, Have a nice day!]"
2,thanks,"[thank you, thanks, thanks for help, that's he...","[Happy to help!, You're welcome!, Feel free to..."
3,nothing,[],"[Please provide more context, Not sure I under..."
4,options,"[How you could help me?, What you can do?, Wha...",[I am a data science chatbot. My capabilities ...
5,jokes,"[Tell me a joke, Joke, Make me laugh, tell me ...",[A perfectionist walked into a bar...apparentl...
6,Identity,"[Who are you, what are you]",[I am a chatbot that was made to provide you w...
7,programmer,"[Who made you, who designed you, who programme...","[I was made by Zaid and Layan., My creators ar..."
8,activity,"[what are you doing, what are you upto]","[Talking to you, of course!, Just chatting wit..."
9,contact,"[contact developer, contact layan, contact pro...",[You can contact my creators at theirs Linkedi...
